In [17]:
import yfinance as yf
import pandas as pd
import polars as pl
import os
import re
import urllib.parse
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math 
import pyarrow
import numpy as np
from io import StringIO
import time
import logging
from linearmodels.panel import PanelOLS

Collect scores from hard memory


In [18]:
root = r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\Analysis\data\snp_500_scores'

ticker_symbol_pattern = re.compile(r'^([A-Z]+(?:\.[A-Z]+)?)\.parquet$')

df_dict_scores = {}

for dir in os.listdir(root):
    
    match = ticker_symbol_pattern.search(dir)

    ticker = match.group(1)
    ticker = ticker.replace('.', '-')
    print(ticker)

    file_path = os.path.join(root, dir)

    df = pl.read_parquet(file_path)
    df_dict_scores[ticker] = df 


A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FOXA
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
IN

In [19]:
df_dict_scores['ZTS']

2019-02-14,2019-05-02,2019-08-06,2019-11-07,2020-02-13,2020-05-06,2020-08-06,2020-11-05,2021-02-16,2021-05-06,2021-08-05,2021-11-04,2022-02-15,2022-05-05,2022-08-04,2022-11-03,2023-02-14,2023-05-04,2023-08-08,2023-11-02
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.304808,-0.341513,-0.302144,-0.279245,-0.321545,-0.371341,-0.360852,-0.352941,-0.317005,-0.391813,-0.385656,-0.37234,-0.311451,-0.41853,-0.400593,-0.396226,-0.29117,-0.296748,-0.285171,-0.26776
-634.0,-167.0,-155.0,-148.0,-691.0,-241.0,-271.0,-276.0,-906.0,-268.0,-285.0,-280.0,-854.0,-262.0,-270.0,-273.0,-709.0,-146.0,-150.0,-147.0
0.449143,0.486296,0.46963,0.456949,0.448631,0.440661,0.419873,0.42329,0.41379,0.419026,0.408294,0.412958,0.419573,0.42563,0.412292,0.406832,0.439736,0.463665,0.45339,0.448852
4036.0,1242.0,1268.0,1279.0,4096.0,1333.0,1517.0,1603.0,4879.0,1317.0,1398.0,1466.0,4716.0,1199.0,1288.0,1310.0,4535.0,1174.0,1284.0,1290.0
52917.0,15738.0,16749.0,17448.0,53506.0,18483.0,22313.0,23250.0,67935.0,18912.0,21023.0,21537.0,64740.0,16870.0,18891.0,19459.0,59370.0,15299.0,17288.0,17537.0


This is support func to find end prices and their dates. It takes as an arguments starting index, which is the nuber of row of hist dataframe and this hist dataframe itself. Hist dataframe contain daily OHLC data for particular ticker (company). 

It also cheks wether index of end price is in hist, meaning that if this particular report was released very resently (less then 8 trading days ago) it will append none values for end prices out of range of hist df instead of raising an error.

It returns lists of end prices and corresponding dates (none values for prices and their dates, if they are out of range of hist df).

In [20]:
def find_end_price(start_index, hist):
    end_price_list = []
    end_price_date_list = []
    
    for x in range(2, 8):
        idx = start_index + x
        if idx < len(hist):
            end_price_list.append(hist.iloc[idx]['Open'])
            end_price_date_list.append(hist.index[idx])
        else:
            end_price_list.append(None)
            end_price_date_list.append(None)
    return end_price_list, end_price_date_list

This is support func to calc the regular returns. It takes start price and end_price_list as an arguments. It checks whether value in end price list is none and if it is, it appends none in returns list, instead of raising an error. So we are shure that all end price lists are of the same length, but for those prices that are not found in historical data we have none values for returns.

In [21]:
def regular_returns(start_price, end_price_list):
    retuns = []
    for end_price in end_price_list:
        if end_price is None:
            retuns.append(None)
        else:
            ret = (end_price - start_price) / start_price * 100
            retuns.append(ret)
    return retuns

This is the support func that calc SnP500 returns. It takes_start date and end_price_date_list as and arguments and then calculate returns for this timeframes. This func also checks whether value of end_price_date_list is none (meaning that it is not found in historical data), and if it is, func will append none instead of raising an error.

Now we are shure that list of snp_returns will be of equal length for all report release dates and all time frames, but for those returns that cannot be calculates due to non existent end prices we will have nones 

In [22]:
def snp_500_return(start_date, end_price_date_list, snp_price):

    snp_returns = []

    for end_date in end_price_date_list:
        if end_date is None:
            snp_returns.append(None)

        else: 
            start_index = snp_price.index.get_loc(start_date)
            start_price = snp_price.iloc[start_index]['Open']
    
            end_index = snp_price.index.get_loc(end_date)
            end_price = snp_price.iloc[end_index]['Open']
    
            ret = (end_price - start_price) / start_price * 100
            
            snp_returns.append(ret)

    return snp_returns

This is suppurt function that retrive the end quarter price. It take as an argument hist - historical price dataframe, date_str - the date of release of this particular report, df - company dataframe, with cols named with dates of releases of corresponding reports.

It takes the release date for current report and checks whether it is the last date in time series or not. If it is (this mean that we are are realy close to the end of hist df) we consider the end of hist df as the end of quarter. if it is not, we take next date in company df (which is the date of releas of the next quarterly report) and see its open price, this is pricisely the end of the quarter.

In [23]:
def find_end_quarter(hist, date_str, start_date, df):

    col_names = df.columns
    current_date_index = col_names.index(date_str)

    if current_date_index < len(col_names) - 1:

        next_date = col_names[current_date_index + 1]

        next_date_time_stemp = pd.Timestamp(next_date, tz='America/New_York')

        while next_date_time_stemp not in hist.index:

            next_date_time_stemp += pd.Timedelta(days=1)
    
        end_quarter_index = hist.index.get_loc(next_date_time_stemp)
        end_quarter_price = hist.iloc[end_quarter_index]['Open']
        end_quarter_date = hist.index[end_quarter_index]

        start_quarter_index = hist.index.get_loc(start_date)

        quarter_length = end_quarter_index - start_quarter_index

    else:
        end_quarter_price = None
        end_quarter_date = None
        quarter_length = None
    

    return end_quarter_price, end_quarter_date, quarter_length

This is a func that takes as an input a key-value pair from df_dict_scores and calculate 2-days, 3-days, 4-days, 5-days, 6-days, 7-days and full quarter excess returns for it (S&P500 is the benchmark). It stores the result as a polars df with dates of starting of the period as col names and corresponding returns as col values (seven in each: starting from 2-days down to 7-days and full quarter) - 

In [24]:
def computing_returns(tic_sym, df):

    company_ticker = yf.Ticker(tic_sym)
    hist = company_ticker.history(period="max")

    snp500_ticker = yf.Ticker("^GSPC")
    snp_price = snp500_ticker.history(period="max", auto_adjust=True)
    
    returns = {}

    for date_str in df.columns:

        start_date = pd.Timestamp(date_str, tz='America/New_York')
        
        while start_date not in hist.index:

            start_date += pd.Timedelta(days=1)
        

        start_index = hist.index.get_loc(start_date)

        start_price = hist.iloc[start_index]['Open']

        end_price_list, end_price_date_list = find_end_price(start_index, hist)
        
        if None not in end_price_list:
            end_quarter_price, end_quarter_date, length_of_quarter = find_end_quarter(hist, date_str, start_date, df)
            
        else:
            end_quarter_price = None
            end_quarter_date = None

        
        end_price_list.append(end_quarter_price)
        end_price_date_list.append(end_quarter_date)
        
        reg_returns = regular_returns(start_price, end_price_list)
        snp_returns = snp_500_return(start_date, end_price_date_list, snp_price)
        
        excess_returns = [a - b if a is not None and b is not None else None for a, b in zip(reg_returns, snp_returns)]
        
        #normalization: divide returns for each time frame by number of trading days (2-days returns/2, 3-days returns/3, ful quarter returns/length_of_quarter)
        timeframe_length = [2, 3, 4, 5, 6, 7, length_of_quarter]

        normalized_excess_returns = [x / y if x is not None and y is not None else None for x, y in zip(excess_returns, timeframe_length)]

        returns[date_str] = normalized_excess_returns

    returns = pl.DataFrame(returns)
    
    return returns


In [143]:
#example ticker symbol and correponding df
tic_sym = "EMR" #A CCI ETR DXCM HPE BAC EMR
df = df_dict_scores[tic_sym]
df

2019-02-06,2019-05-09,2019-08-07,2019-11-18,2020-02-05,2020-04-24,2020-08-05,2020-11-16,2021-02-03,2021-05-05,2021-08-04,2021-11-15,2022-02-02,2022-05-04,2022-08-09,2022-11-14,2023-02-08,2023-05-03,2023-08-02,2023-11-13
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.217949,-0.202381,-0.238636,-0.230303,-0.384615,-0.459716,-0.46281,-0.2044,-0.33913,-0.328767,-0.228571,-0.181395,-0.234375,-0.239264,-0.384615,-0.199009,-0.451429,-0.386935,-0.428571,-0.269643
-34.0,-34.0,-42.0,-228.0,-40.0,-97.0,-112.0,-223.0,-39.0,-48.0,-32.0,-195.0,-30.0,-39.0,-75.0,-241.0,-79.0,-77.0,-84.0,-302.0
0.437838,0.389349,0.403429,0.488596,0.432056,0.292865,0.262846,0.494217,0.447587,0.372603,0.382313,0.515262,0.465734,0.382932,0.357692,0.527428,0.464082,0.492218,0.52381,0.521252
324.0,329.0,353.0,2228.0,248.0,275.0,266.0,2521.0,269.0,272.0,281.0,2549.0,333.0,350.0,372.0,2971.0,407.0,506.0,572.0,2698.0
5436.0,6418.0,6649.0,26107.0,4067.0,6579.0,6633.0,29291.0,4342.0,5366.0,5596.0,28709.0,5220.0,6625.0,7840.0,33437.0,6732.0,8082.0,8652.0,31669.0


In [144]:
computing_returns(tic_sym, df)

2019-02-06,2019-05-09,2019-08-07,2019-11-18,2020-02-05,2020-04-24,2020-08-05,2020-11-16,2021-02-03,2021-05-05,2021-08-04,2021-11-15,2022-02-02,2022-05-04,2022-08-09,2022-11-14,2023-02-08,2023-05-03,2023-08-02,2023-11-13
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.002342,0.297828,-0.653368,-0.070914,0.068036,1.856722,1.40851,-0.457032,0.366098,1.768283,2.031593,-0.703504,2.140717,2.458773,-0.981014,0.855814,-0.621982,-0.445915,-0.491932,-0.03301
0.176652,0.28006,-0.81048,0.005011,-0.442912,2.152086,1.343315,0.13991,0.722099,1.755969,1.090839,-1.217779,0.948349,1.10745,-0.794469,0.237964,-0.394866,-0.243924,-0.319638,-0.221751
0.292629,-0.070384,-0.997705,0.236673,-0.552622,1.101761,2.297626,0.07033,0.386174,1.477176,0.860043,-0.944249,0.966173,0.698054,-0.650291,0.424666,-0.497063,-0.600124,-0.26501,-0.176392
0.297011,0.157382,-0.794891,0.235414,-0.287636,0.472874,2.247784,0.027241,0.221853,1.430222,0.980487,-0.832022,0.899692,0.49432,-0.569829,-0.129876,-0.444686,-0.396799,-0.14841,0.161123
0.19139,-0.051524,-0.594908,0.196532,-0.150439,-0.119903,1.834242,0.405925,0.108073,1.085925,0.995911,-0.504243,0.620911,0.455973,-0.379172,0.186257,-0.364751,-0.896542,-0.287397,-0.139518
-0.015635,-0.061854,-0.578416,0.123609,-0.429957,0.226748,1.342296,0.246367,-0.011746,1.131299,0.828215,-0.497123,0.688295,0.315935,-0.198871,0.132983,-0.19063,-0.522837,-0.107212,-0.142737
-0.094277,-0.1348,0.214857,-0.087865,-0.224689,0.016,0.215342,0.029786,-0.011877,0.058834,-0.072378,-0.015933,-0.001639,0.051517,0.169238,-0.192297,-0.039085,0.079133,-0.125107,null


This func calculate suprise variable. It takes ticker, and corresponding df, uses yfinance to retrive surprise value (in persentage points) and stack it as a row, to the bottom of the df. In case of an error it returns none and false.

In [147]:
def calc_suprise(tic_sym, df, period=10):
    periods = [50, 60, 70, 100, 1000]

    initial_delay = 1
    max_delay = 32

    eps = None

    failed_dates = 0

    delay = initial_delay
    for period in periods:
        
        try:
            company_ticker = yf.Ticker(tic_sym)
            eps = company_ticker.get_earnings_dates(limit=period)
            eps_reset = eps.reset_index()

            duplicates_mask = eps_reset.duplicated(keep=False)
            eps_reset = eps_reset[~duplicates_mask]

            eps = eps_reset.set_index(eps.index.name)
            break
            
        except Exception as e:
            logging.error(f"Error calculating surprise for {tic_sym}: {e}")
        
        time.sleep(delay)
        delay = min(delay * 2, max_delay)
        
    if eps is None:
        return None, False, failed_dates
    
    else:
        available_indices = list(eps.index)
        #print(available_indices)

        eps_dict = {}

        for col in df.columns:

            if available_indices:
                
                date_of_rep = pd.Timestamp(col, tz='America/New_York')
                closest_index = min(available_indices, key=lambda x: abs(x - date_of_rep))

                available_indices = [dt for dt in available_indices if dt < closest_index]

                eps_surprise = eps.loc[closest_index, 'Surprise(%)']

                eps_dict[col] = eps_surprise
            else:
                failed_dates += 1
                eps_dict[col] = None

        eps_df = pl.DataFrame(eps_dict)   
        stacked_df = df.vstack(eps_df) 

        return stacked_df, True, failed_dates

In [148]:
calc_suprise(tic_sym, df)

ERROR:root:Error calculating surprise for EMR: 'Earnings Date'
ERROR:root:Error calculating surprise for EMR: 'Earnings Date'
ERROR:root:Error calculating surprise for EMR: 'Earnings Date'
ERROR:root:Error calculating surprise for EMR: 'Earnings Date'
ERROR:root:Error calculating surprise for EMR: 'Earnings Date'


(None, False, 0)


This is an alternative function to calculate eps_srprise. In case of calc_suprise() failed to retrive it, this function attempts to retrive at least some values of srprise. In case of fail of this function it just fill null values instead.

In [29]:
def calc_suprise_sec_attempt(tic_sym, df):
    eps_dict = {}

    try:
        company_ticker = yf.Ticker(tic_sym)
        eps = company_ticker.earnings_dates

        available_indices = list(eps.index)

        eps_dict = {}

        retrived_dates = 0

        for col in sorted(df.columns, reverse=True):

            if available_indices:
                date_of_rep = pd.Timestamp(col, tz='America/New_York')

                closest_index = min(available_indices, key=lambda x: abs(x - date_of_rep))

                available_indices = [dt for dt in available_indices if dt < closest_index]

                eps_surprise = eps.loc[closest_index, 'Surprise(%)']

                eps_dict[col] = eps_surprise

                retrived_dates += 1
            else:
                eps_dict[col] = None

        failed_dates = len(df.columns) - retrived_dates

        eps_dict = {key: eps_dict[key] for key in sorted(eps_dict)}

    except Exception as e:
        logging.error(f"Error for second attempt surprise for {tic_sym}: {e}")

        for col in df.columns:
            eps_dict[col] = None

        failed_dates = len(df.columns)

    eps_df = pl.DataFrame(eps_dict)   
    stacked_df = df.vstack(eps_df)

    return stacked_df, failed_dates

calc_suprise_sec_attempt(tic_sym, df)

(shape: (6, 20)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ 2019-02-2 ┆ 2019-04-2 ┆ 2019-07-2 ┆ 2019-10-2 ┆ … ┆ 2023-02-2 ┆ 2023-05-0 ┆ 2023-07-3 ┆ 2023-10- │
 │ 6         ┆ 6         ┆ 9         ┆ 8         ┆   ┆ 2         ┆ 1         ┆ 1         ┆ 31       │
 │ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
 │ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ -0.498493 ┆ -0.366154 ┆ -0.349896 ┆ -0.354011 ┆ … ┆ -0.484312 ┆ -0.333903 ┆ -0.355253 ┆ -0.34364 │
 │           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 8        │
 │ -1819.0   ┆ -357.0    ┆ -338.0    ┆ -331.0    ┆ … ┆ -1914.0   ┆ -391.0    ┆ -443.0    ┆ -422.0   │
 │ 0.49479   ┆ 0.575429  ┆ 0.580203  ┆ 0.57295   ┆ … ┆ 0.499189  ┆

This is the general loop, that takes each key-value pair from df_dict_scores, applies computing_returns() to it, then verticaly stackes the result with orig df and stores it in the new dict

In [173]:
dict_for_anal = {}
row_names = pl.Series("row_names", ["lm_polarity", "lm_raw", "hiv4_polarity", "hiv4_raw", "doc_length", "eps_surprise", "2_day_reterns","3_day_reterns", "4_day_reterns", "5_day_reterns", "6_day_reterns", "7_day_reterns", "full_quarter_returns"])

company_fails = 0
doc_fails = 0
total_docs = 0

for tic_sym, df in df_dict_scores.items():
    total_docs += df.shape[1]
    print(tic_sym)
    
    regressors_df, sucssess, failed_dates = calc_suprise(tic_sym, df)

    if failed_dates > 0:
        doc_fails += failed_dates
        company_fails += 1

    if sucssess == False:
        company_fails +=1
        regressors_df, failed_dates = calc_suprise_sec_attempt(tic_sym, df)
        doc_fails += failed_dates

    returns_df = computing_returns(tic_sym, df)
    
    stacked_df = regressors_df.vstack(returns_df)
    stacked_df_with_indx = stacked_df.hstack([row_names])
    
    dict_for_anal[tic_sym] = stacked_df_with_indx

print(f'Company fails ratio: {company_fails/len(df_dict_scores)}')
print(f'Documents fails ratio: {doc_fails/total_docs}')


A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC


ERROR:root:Error calculating surprise for CNC: 'Earnings Date'
ERROR:root:Error calculating surprise for CNC: 'Earnings Date'
ERROR:root:Error calculating surprise for CNC: 'Earnings Date'
ERROR:root:Error calculating surprise for CNC: 'Earnings Date'
ERROR:root:Error calculating surprise for CNC: 'Earnings Date'
ERROR:root:Error for second attempt surprise for CNC: 'Earnings Date'


CNP


ERROR:root:Error calculating surprise for CNP: 'Earnings Date'
ERROR:root:Error calculating surprise for CNP: 'Earnings Date'
ERROR:root:Error calculating surprise for CNP: 'Earnings Date'


COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN


...

UAL
UBER
UDR
UHS
ULTA
UNH
UNP
UPS
URI
USB
V
VICI
VLO
VLTO
VMC
VRSK
VRSN
VRTX
VST
VTR
VTRS
VZ
WAB
WAT
WBA
WBD
WDC
WEC
WELL
WFC
WM
WMB
WMT
WRB
WRK
WST
WTW
WY
WYNN


ERROR:root:Error calculating surprise for WYNN: 'Earnings Date'
ERROR:root:Error calculating surprise for WYNN: 'Earnings Date'
ERROR:root:Error calculating surprise for WYNN: 'Earnings Date'
ERROR:root:Error calculating surprise for WYNN: 'Earnings Date'
ERROR:root:Error calculating surprise for WYNN: 'Earnings Date'
ERROR:root:Error for second attempt surprise for WYNN: 'Earnings Date'


XEL


ERROR:root:Error calculating surprise for XEL: 'Earnings Date'
ERROR:root:Error calculating surprise for XEL: 'Earnings Date'
ERROR:root:Error calculating surprise for XEL: 'Earnings Date'


XOM
XYL
YUM
ZBH
ZBRA
ZTS
Company fails ratio: 0.33400402414486924
Documents fails ratio: 0.10193285859613428


In [150]:
dict_for_anal["AMGN"]

2019-02-13,2019-05-01,2019-07-31,2019-10-30,2020-02-12,2020-05-01,2020-07-29,2020-10-29,2021-02-09,2021-04-28,2021-08-04,2021-11-03,2022-02-16,2022-04-28,2022-08-05,2022-11-04,2023-02-09,2023-04-28,2023-08-04,2023-10-31,row_names
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
-0.381314,-0.582022,-0.561997,-0.478261,-0.429706,-0.49076,-0.503546,-0.447323,-0.370518,-0.456693,-0.411215,-0.410029,-0.355102,-0.445,-0.434343,-0.435606,-0.339513,-0.466227,-0.524116,-0.447293,"""lm_polarity"""
-853.0,-259.0,-349.0,-231.0,-1082.0,-239.0,-284.0,-259.0,-1116.0,-232.0,-264.0,-278.0,-1044.0,-178.0,-258.0,-230.0,-989.0,-283.0,-326.0,-314.0,"""lm_raw"""
0.477493,0.463387,0.432812,0.493083,0.472367,0.520228,0.5,0.493216,0.490546,0.462989,0.460759,0.468237,0.490217,0.45591,0.444444,0.476099,0.479358,0.449772,0.441495,0.468189,"""hiv4_polarity"""
4381.0,829.0,1079.0,1105.0,4530.0,1093.0,1154.0,1236.0,5604.0,1007.0,1227.0,1312.0,5587.0,972.0,1248.0,1245.0,5469.0,1182.0,1181.0,1435.0,"""hiv4_raw"""
58584.0,12849.0,17391.0,15944.0,60897.0,14698.0,16226.0,17354.0,71733.0,15393.0,18382.0,19640.0,71201.0,14811.0,19390.0,18517.0,70718.0,17953.0,18812.0,20885.0,"""doc_length"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-0.338307,0.295731,1.437831,0.821738,-0.612361,0.338548,-0.927999,0.495241,-0.135079,0.631625,-1.178935,-0.400512,0.645617,-0.090017,-0.111424,2.45089,0.274592,-0.402221,2.63151,0.297404,"""4_day_reterns"""
-0.36451,0.275983,1.382592,0.649341,-0.467425,-0.109637,-1.077053,0.612635,-0.312587,0.43361,-0.893221,0.053664,0.940166,-0.246152,-0.36322,0.987209,0.096159,-0.13387,2.280248,0.40057,"""5_day_reterns"""
-0.350749,0.126846,0.984435,0.593687,-0.361093,-0.264354,-1.131169,0.308994,-0.029872,0.599212,-0.824795,0.147059,0.515549,0.091953,-0.44508,0.514661,0.012818,-0.133824,1.909163,0.225427,"""6_day_reterns"""


This func takes as input key-value pair (company name and dataframe with sentiment scores and returns), regress each retern (2_days, 3_days, etc) on polarity score and raw sentiment score. Then it collects the slopes (beta-coefficient) of returns regressed on each score in the separate polars series (in the loop they will be staked in dataframe)

In [155]:
def add_quarter_dummy(company_df_pd, var_names):
    
    dates = pd.to_datetime(company_df_pd.columns)
   
    date_series = pd.Series(dates.quarter, index=dates)
   
    dummies = pd.get_dummies(date_series, prefix="Q")
    dummies.index = dummies.index.strftime('%Y-%m-%d')
    
    all_quarters = ["Q_1", "Q_2", "Q_3", "Q_4"]
    dummies = dummies.reindex(columns=all_quarters, fill_value=False)
   
    dummies = dummies.drop(columns=["Q_4"])
    
    dummies = dummies.transpose()
    df_wit_dum = pd.concat([company_df_pd, dummies])
    
    df_wit_dum = df_wit_dum.transpose()
   
    for col in df_wit_dum.columns:
        df_wit_dum[col] = pd.to_numeric(df_wit_dum[col])

    boolean_cols = ['Q_1', 'Q_2', 'Q_3']
    df_wit_dum[boolean_cols] = df_wit_dum[boolean_cols].astype(int)

    res_col_names = var_names + boolean_cols

    df_wit_dum.columns = res_col_names

    return df_wit_dum    

In [156]:
def fit_regression(y, x):

    x = sm.add_constant(x)
    model = sm.OLS(y, x, missing='drop').fit()
    
    slope = model.params.iloc[1]
    
    return slope

In [157]:
def regression_slopes(df_wit_dum, X_names, company_name):
    
    for col in df_wit_dum.select_dtypes(include=['bool']).columns:
        df_wit_dum[col] = df_wit_dum[col].astype(int)
        
    df_wit_dum = df_wit_dum.apply(pd.to_numeric, errors='coerce')

    # x in range(0, 4) represents set of independent variables; 4th column represents control variable for document length; -3, -2, -1 columns represent dummy variables for quarters
    
    X = [df_wit_dum.iloc[:, [x, 4, 5, -3, -2, -1]] for x in range(0, 4)]

    Y = [df_wit_dum.iloc[:, [y]] for y in range(6, 13)]

    slopes_dict = {}

    for i, x in enumerate(X):

        name_of_score = X_names[i][0].split()[0]
        slopes_name = f'{name_of_score}_slopes'
        
        slopes = []
        for y in Y:
    
            slope = fit_regression(y, x)
            slopes.append(slope)

        slopes_dict[slopes_name] = pl.Series(company_name, slopes)
    
    return slopes_dict

This is the func checks if there is enough data in single company dataframe. As an argument it takes a company df with out all dummies and check that each col contaian enough data to fit the model. 

In [158]:
def check_6_nonnull(df):
    df = df.transpose()

    counts = df.count()
    mask = counts < 6
    if mask.sum_horizontal().item() > 0:
        return False
    else:
        return True

This is the general loop that takes dict_for_anal whith scores and returns and for each company in the dict it calculates dict with four dfs: first one contain regression slopes for lm_polarity scores, second one for lm_raw scores and so on for each regressor. Serieses stacked in the separete dataframes (for raw and for polarity scores) 

Note: sector dummies are not included because each regression estimated for a single company (and it obviosly has the same sector variable value)

In [159]:
def compute_all_slopes(dict_for_anal):

    slopes_df_dict = {
        "lm_polarity_slopes_df" : pl.DataFrame(),
        "lm_raw_slopes_df" : pl.DataFrame(),
        "hiv4_polarity_slopes_df" : pl.DataFrame(),
        "hiv4_raw_slopes_df" : pl.DataFrame()
    }

    fails = 0

    for company_name, company_df in dict_for_anal.items(): 

        X_names = [company_df.select(pl.col("row_names")).row(x) for x in range(0, 4)]

        var_names = company_df.get_column("row_names").to_list()
        company_df_woi = company_df.select(pl.exclude("row_names"))

        company_df_woi = company_df_woi.to_pandas()
        print(company_name)

        df_wit_dum = add_quarter_dummy(company_df_woi, var_names)

        enough_data = check_6_nonnull(company_df)

        if enough_data == True:

            slopes_dict = regression_slopes(df_wit_dum, X_names, company_name)
    
            for x in X_names:
                
                name_of_score = x[0].split()[0]
                slopes_name = f'{name_of_score}_slopes'
                
                df_name = f"{name_of_score}_slopes_df"
                
                slopes_df_dict[df_name] = slopes_df_dict[df_name].hstack([slopes_dict[slopes_name]])
        else:
            fails +=1
    print(f'fails rate: {fails/len(dict_for_anal)}')

    return slopes_df_dict
        

In [160]:
slopes_df_dict = compute_all_slopes(dict_for_anal)

A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FOXA
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
IN

As an output from previous functions we have dataframes with regression slopes of different returns on raw, polarity scores and Loughran-McDonald and Harvard dictionaries. Each col of these dfs contain regression slopes for particular company. First row contain slope for 2_days return, second row contain slope for 3_days return, etc.  I have decided to check what is the proportion of positive regression slopes for each return timeframe (2_days, 3_days, etc). For that purpose i will transpose each of these dfs, apply condition > 0, then verticaly sum boolean values and obtain the prorortion of positive slopes for each time frame. 

In [161]:
def calc_efficiency_metrics(slopes_df):

    df_tranposed = slopes_df.transpose()

    res = df_tranposed.select(pl.all() > 0).sum()
    total_num_of_slopes = slopes_df.shape[1]

    positive_slopes = [res.select(pl.col(res.columns[x]).gather(0)).item() for x in range(slopes_df.shape[0])]

    for counter, x in enumerate(positive_slopes):
        print(f'Prorortion of positive {counter + 2}_day returns: {x/total_num_of_slopes}')

This loop apply calc_efficiency_metrics function to each item of slopes_df_dict where each item is a dataframe which regression slopes for each regressor.

Note: actually there is no 8_day returns, it is actually full-quarter returns (just easier to print it programmatically)

In [162]:
for key, slopes_df in slopes_df_dict.items():
    print("\n")
    print(f'Metrics for: {key}')
    calc_efficiency_metrics(slopes_df)



Metrics for: lm_polarity_slopes_df
Prorortion of positive 2_day returns: 0.47844827586206895
Prorortion of positive 3_day returns: 0.49137931034482757
Prorortion of positive 4_day returns: 0.4870689655172414
Prorortion of positive 5_day returns: 0.5150862068965517
Prorortion of positive 6_day returns: 0.5
Prorortion of positive 7_day returns: 0.49137931034482757
Prorortion of positive 8_day returns: 0.4525862068965517


Metrics for: lm_raw_slopes_df
Prorortion of positive 2_day returns: 0.4676724137931034
Prorortion of positive 3_day returns: 0.4827586206896552
Prorortion of positive 4_day returns: 0.48060344827586204
Prorortion of positive 5_day returns: 0.4849137931034483
Prorortion of positive 6_day returns: 0.48060344827586204
Prorortion of positive 7_day returns: 0.4827586206896552
Prorortion of positive 8_day returns: 0.46336206896551724


Metrics for: hiv4_polarity_slopes_df
Prorortion of positive 2_day returns: 0.46551724137931033
Prorortion of positive 3_day returns: 0.48275

Positive slopes means that sentiment scores and returns mooves in the same direction (hense, they are working as predictors of stock price movement). As the result of this preliminary analysis we can say that sentiment scores of quarterly reports are not very good predictors of stock movments. Positive proportion of this slopes are close to 0.5, which means that on average in our sample there are 50/50 chance that scores and returns are mooving in the same direction.

This is the separate function for regression fitting, that will be used to fit regression for all companies together. Returns has no autocorrelation, but very likely exhibit heteroscedasticity. To handle this problem White's SE are used to obtain pobust estimation of p_val

In [163]:
def fit_rob_se(y, x):

    x = sm.add_constant(x)
    model = sm.OLS(y, x, missing='drop').fit()
    robust_se = model.get_robustcov_results(cov_type='HC0')
    
    return robust_se.summary()

This function is used to get parametrs of the model and conviniently display them. As an argument it takes regression model summary and returns a table only wit significant coefficients. Since we have 4 different sets of regressors and 7 dependent variables, in total we have 28 regressions. So, with help of this function, only those parametrs with p_val < 5% will be displaied as an output of regression model fit. 

In [164]:
def extract_params(summary):
    
    coef_table = summary.tables[1].as_html()
    coef_table_io = StringIO(coef_table)
    df = pd.read_html(coef_table_io, header=0, index_col=0)[0]

    df.columns = ['coef', 'std err', 't', 'P>|t|', '[0.025', '0.975]']
    
    df = df.reset_index()
    df = df.rename(columns={'index': 'Variable'})

    filtered_df = df[df['P>|t|'] < 0.05]
    
    return filtered_df


This is function is used to regress dependent variables on all set of regressors using all dummies (sector dummies included). It takes total df and variadle manes as an input, and prints significant coefficients as an output for each regression.

In [165]:
def regression_single_slope(df_wit_dum, X_names, Y_names):

    for col in df_wit_dum.select_dtypes(include=['bool']).columns:
        df_wit_dum[col] = df_wit_dum[col].astype(int)

    df_wit_dum = df_wit_dum.apply(pd.to_numeric, errors='coerce')

    # x in range(0, 4) represents set of independent variables; 4th column represents control variable for document length; 5th column represent control variable for eps surprize in persentage points; from -14 to -1 columns represent dummy variables for quarters and economic sector.
    X = [df_wit_dum.iloc[:, [x, 4, 5, -14, -13, -12, -11, -10, -9,-8 ,-7, -6, -5 ,-4, -3, -2, -1]] for x in range(0, 4)]

    Y = [df_wit_dum.iloc[:, [y]] for y in range(6, 13)]

    for i, x in enumerate(X):

        name_of_score = X_names[i][0]

        for j, y in enumerate(Y):
            returns_timeframe = Y_names[j][0]
            
            summary = fit_rob_se(y, x)
            params_df = extract_params(summary)
            if not params_df.empty:
                print(f' X variable: {name_of_score}', '\n', f'Y variable: {returns_timeframe}', '\n')
                print(f'Significant coefficients: \n {params_df}', '\n\n\n') 

This function adds a sector value to data. As an input it takes a dataframe and company name. It uses yfinance library to extract sector value and add it as a column of categorical values to a df.

In [166]:
def addsector_value(company_name, df_wit_dum):
    
    ticker = yf.Ticker(company_name)
    company_info = ticker.info

    sector = company_info.get('sector', 'Sector information not available')
    df_wit_dum['sector'] = sector

    return df_wit_dum
    

This func stackes df of different companies in one df, apply get_dummies metod, to transform categorical data to actuall dummy variable set and then stores the result in the corresponding variables.

In [167]:
def compute_all_var_set(dict_for_anal):

    df_list = []
    
    for company_name in dict_for_anal:


        X_names = [dict_for_anal[company_name].select(pl.col("row_names")).row(x) for x in range(0, 4)]
        Y_names = [dict_for_anal[company_name].select(pl.col("row_names")).row(x) for x in range(6, 13)] 

        var_names = dict_for_anal[company_name].get_column("row_names").to_list()

        company_df_woi = dict_for_anal[company_name].select(pl.exclude("row_names"))
        company_df_pd = company_df_woi.to_pandas(use_pyarrow_extension_array=True)

        df_wit_dum = add_quarter_dummy(company_df_pd, var_names)
        df_wit_sector = addsector_value(company_name, df_wit_dum)

        df_list.append(df_wit_sector)
        
    total_df = pd.concat(df_list, ignore_index=True)
    total_df_with_dummies = pd.get_dummies(total_df, columns=['sector'])
    
    return total_df_with_dummies, X_names, Y_names


In [168]:
total_df_with_dummies, X_names, Y_names = compute_all_var_set(dict_for_anal)

This is the result of fitting regression on total data set. Almost in all cases only covariate variables has significant coeffisients. There are to exceptions: 
1. hiv4_polarity score for full_quarter_returns has coefficient -0.0751 with P>|t| 0.045.
    Negative sign suggests that this score moves in the opposit derection with full_quarter_returns (the opposit is expected).
    P_val is barely less 5% level (4.5%). Despite it's significants this is more likely to be a type one error.
2. hiv4_raw score for full_quarter_returns has coefficient -0.000020 with P>|t| 0.015.
    Slope has tiny ablosute value, so it is economicly insignificant, and would be regected at 1% significance level.

In [169]:
regression_single_slope(total_df_with_dummies, X_names, Y_names)

 X variable: lm_polarity 
 Y variable: 2_day_reterns 

Significant coefficients: 
             Variable      coef   std err       t  P>|t|    [0.025    0.975]
2         doc_length  0.000004  0.000001   3.318  0.001  0.000002  0.000007
3       eps_surprise  0.000087  0.000004  21.308  0.000  0.000078  0.000095
4                Q_1 -0.192300  0.063000  -3.048  0.002 -0.316000 -0.069000
5                Q_2 -0.206700  0.058000  -3.545  0.000 -0.321000 -0.092000
6                Q_3 -0.172400  0.058000  -2.998  0.003 -0.285000 -0.060000
17  sector_Utilities -0.219800  0.059000  -3.714  0.000 -0.336000 -0.104000 



 X variable: lm_polarity 
 Y variable: 3_day_reterns 

Significant coefficients: 
                   Variable      coef       std err       t  P>|t|    [0.025  \
2               doc_length  0.000003  9.920000e-07   3.201  0.001  0.000001   
3             eps_surprise  0.000077  2.960000e-06  25.853  0.000  0.000071   
4                      Q_1 -0.128400  4.900000e-02  -2.643  0

Fixed effects 

This func is converting date strings in quarter strings 

In [139]:
def convert_to_quarter(date_str):
    date = datetime.strptime(date_str, '%Y-%m-%d')
    quarter = (date.month - 1) // 3 + 1
    formatted_string = f'{date.year}.{quarter}'
    return float(formatted_string)


In [34]:

dict_for_anal['A']

2019-03-05,2019-05-30,2019-08-30,2019-12-19,2020-03-03,2020-06-01,2020-09-01,2020-12-18,2021-03-02,2021-06-01,2021-09-01,2021-12-17,2022-03-03,2022-05-31,2022-09-01,2022-12-21,2023-03-03,2023-05-26,2023-08-31,2023-12-20,row_names
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
-0.329588,-0.321787,-0.32539,-0.266055,-0.322245,-0.346789,-0.34978,-0.301829,-0.361823,-0.362896,-0.36022,-0.290069,-0.363636,-0.384206,-0.376554,-0.276257,-0.38737,-0.391225,-0.418895,-0.33429,"""lm_polarity"""
-352.0,-353.0,-355.0,-493.0,-310.0,-378.0,-397.0,-594.0,-381.0,-401.0,-393.0,-590.0,-384.0,-433.0,-424.0,-555.0,-411.0,-428.0,-470.0,-699.0,"""lm_raw"""
0.327818,0.318681,0.338225,0.400181,0.369223,0.371591,0.363251,0.412808,0.37644,0.377856,0.375802,0.430503,0.345776,0.345306,0.344507,0.421466,0.339753,0.331607,0.30764,0.396259,"""hiv4_polarity"""
1387.0,1421.0,1521.0,3540.0,1454.0,1703.0,1716.0,3771.0,1601.0,1720.0,1699.0,4076.0,1408.0,1519.0,1521.0,3813.0,1376.0,1409.0,1365.0,3644.0,"""hiv4_raw"""
25966.0,27591.0,27714.0,53169.0,24310.0,28714.0,29581.0,56074.0,26466.0,28792.0,28689.0,58081.0,25613.0,28061.0,27991.0,55213.0,25076.0,26581.0,27194.0,55569.0,"""doc_length"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-0.124922,0.097844,0.415717,0.131847,-0.961461,0.132534,-0.225307,-0.26873,-1.278249,-0.270018,0.399046,0.877456,0.847743,0.189737,0.499011,0.205652,-0.228498,-0.989274,-0.587397,-0.04754,"""4_day_reterns"""
-0.059373,-0.023132,0.473364,0.068141,-0.331229,-0.772055,-0.170479,-0.124213,-0.810767,-0.163926,0.530154,0.776396,0.327516,-0.099889,0.870496,0.267282,-0.04358,-0.858511,-0.862496,0.016178,"""5_day_reterns"""
0.009625,0.126009,0.285714,0.062333,-0.807331,-0.569214,0.113952,-0.217324,-0.482759,-0.069892,0.335702,0.522019,0.673132,-0.11723,0.670713,0.162028,-0.138225,-0.640946,-1.061113,-0.121283,"""6_day_reterns"""


In [136]:
def get_data_for_FE(dict_for_anal):

    total_df = pl.DataFrame()

    for company in dict_for_anal: 
        row_names = dict_for_anal[company].select(pl.col('row_names')).to_series().to_list()
        
        df = dict_for_anal[company].filter(pl.col("row_names") != 'doc_length')
        row_names.remove('doc_length')
        df = df.select(pl.exclude('row_names'))
    
        quarter_strings = [convert_to_quarter(date) for date in df.columns]
        quarter_strings = pl.Series('date', quarter_strings)
    
        df_w_dates = df.transpose().hstack([quarter_strings])
        df_w_company = df_w_dates.with_columns(pl.lit(company).alias('company'))

        column_name_order = ['company', 'date']
        column_name_order.extend(df.transpose().columns)

        prepared_df = df_w_company.select(column_name_order)
        total_df.vstack(prepared_df, in_place=True)

    total_df_pd = total_df.to_pandas()
   
    col_names =  ['company', 'date']
    col_names.extend(row_names)
   
    total_df_pd.columns = col_names
    total_df_pd = total_df_pd.set_index(['company', 'date'])
    
    return total_df_pd 

In [170]:
df_for_FE = get_data_for_FE(dict_for_anal)
print(df_for_FE)

                lm_polarity  lm_raw  hiv4_polarity  hiv4_raw  eps_surprise  \
company date                                                                 
A       2019.1    -0.329588  -352.0       0.327818    1387.0        0.0402   
        2019.2    -0.321787  -353.0       0.318681    1421.0        0.1007   
        2019.3    -0.325390  -355.0       0.338225    1521.0        0.0681   
        2019.4    -0.266055  -493.0       0.400181    3540.0        0.0119   
        2020.1    -0.322245  -310.0       0.369223    1454.0        0.1379   
...                     ...     ...            ...       ...           ...   
ZTS     2022.4    -0.396226  -273.0       0.406832    1310.0        0.1233   
        2023.1    -0.291170  -709.0       0.439736    4535.0        0.1159   
        2023.2    -0.296748  -146.0       0.463665    1174.0        0.1081   
        2023.3    -0.285171  -150.0       0.453390    1284.0           NaN   
        2023.4    -0.267760  -147.0       0.448852    1290.0    

Fuck to show only significant params 

In [171]:
def extract_params_FE(summary):
    coef_table = summary.tables[1].as_html()
    
    coef_table_io = StringIO(coef_table)
    df = pd.read_html(coef_table_io, header=0, index_col=0)[0]
    
    df.columns = ['Parameter', 'Std. Err.', 'T-stat', 'P-value', 'Lower CI', 'Upper CI']

    df = df.reset_index()
    df = df.rename(columns={'index': 'Variable'})

    significant_df = df[df['P-value'] < 0.05]
    
    return significant_df


This is func to fit FE

In [117]:
def fit_FE(df, formula):

    model = PanelOLS.from_formula(formula, data=df)
    #result = model.fit(cov_type='clustered', cluster_entity=True)
    result = model.fit(cov_type='kernel')


    return result.summary

This is func to iterate over all pairs of dep and indep var

In [174]:
def compute_all_FE(df_for_FE):
    
    for x in range(0, 4):
        for y in range(5, 12):
            data = df_for_FE.iloc[:, [y, x, 4]].dropna()

            dep_var = data.columns[0]
            indep_var = data.columns[1]

            formula = f"{dep_var} ~ {indep_var} + eps_surprise + EntityEffects + TimeEffects"
            
            result = fit_FE(data, formula)
            significant_df = extract_params_FE(result)

            if not significant_df.empty:
                print(f' Dependent var: {dep_var} \n Independent var: {indep_var} \n Control var: eps_surprise\n')
                print(significant_df, '\n\n\n')
            
compute_all_FE(df_for_FE)  

 Dependent var: 2_day_reterns 
 Independent var: lm_polarity 
 Control var: eps_surprise

       Variable  Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
1  eps_surprise   0.000068   0.000009  7.2632      0.0   0.00005  0.000087 



 Dependent var: 3_day_reterns 
 Independent var: lm_polarity 
 Control var: eps_surprise

       Variable  Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
1  eps_surprise   0.000064   0.000006  10.012      0.0  0.000051  0.000076 



 Dependent var: 4_day_reterns 
 Independent var: lm_polarity 
 Control var: eps_surprise

       Variable  Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
1  eps_surprise   0.000047   0.000005  10.134      0.0  0.000038  0.000057 



 Dependent var: 5_day_reterns 
 Independent var: lm_polarity 
 Control var: eps_surprise

       Variable  Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
1  eps_surprise   0.000021   0.000003  6.3821      0.0  0.000015  0.000027 



 Dependent var: 6_day_re

jupyter nbconvert --to html Parsim-sec\src\Analysis\excess_ret_500.ipynb